Load libraries

In [35]:
require(ggplot2)
#require(GGally)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
# require(aod)
require(dplyr)

# Load data and set factors

In [36]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$CVS_graph <- factor(mydata$cvs_graph)
mydata$CVS_table <- factor(mydata$cvs_table)
# mydata$pre <- factor(mydata$pre)
# mydata$main <- factor(mydata$main)

In [37]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars,CVS_graph,CVS_table
10127163,L,Concentration,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,L,Width,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Area,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Separation,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Concentration,0,0,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Width,0,0,0,0,1,1,LC,...,0,1,1,1,1,1,1,4,0,0


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"              "CVS_graph"                
[29] "CVS_table"

## Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)
* student attributes: lab_experience, similar_sim, prior_number_virtual_labs, attitude components

We do so using a repeated measures anova where we have repeat measures of "variable" by student.

### Filtering data
**Should we remove students with max pre for a certain variables???**

### Results

In [38]:
am1 <- aov(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am1)


Error: sid
                           Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table                   1  21.10  21.101  31.621 1.01e-07 ***
cvs_graph                   1   2.83   2.825   4.234   0.0415 *  
pre                         1  12.57  12.571  18.839 2.74e-05 ***
lab_experience              1   0.23   0.233   0.350   0.5551    
similar_sim                 1   0.30   0.302   0.452   0.5024    
prior_number_virtual_labs   1   0.20   0.204   0.306   0.5809    
overall.POCC                1   0.34   0.342   0.513   0.4751    
PC1                         1   0.02   0.019   0.029   0.8658    
PC2                         1   0.58   0.584   0.874   0.3514    
Residuals                 137  91.42   0.667                     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table     1   2.84   2.841  12.470 0.000458 ***
cvs_graph     1   4.23   4.231  18.573 2.03e-05 ***
variable      3   

We see that the variance from having repeated measures for each student (first table) has been removed from the overall model error and thus separated in the output.

Looking the second table we notice that both cvs are significant, cvs_graph more significant than cvs_table, variable is significant and so is sim_index.

eta squares, and post hoc test are coming!

## Stat model 2: Predicting transfer data

In [39]:
am2 <- aov(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am2)


Error: sid
                           Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table                   1   0.05  0.0542   0.130 0.7195  
cvs_graph                   1   2.04  2.0383   4.875 0.0289 *
pre                         1   1.58  1.5804   3.780 0.0539 .
lab_experience              1   0.06  0.0646   0.154 0.6949  
similar_sim                 1   0.62  0.6225   1.489 0.2245  
prior_number_virtual_labs   1   2.11  2.1087   5.044 0.0263 *
overall.POCC                1   0.19  0.1877   0.449 0.5040  
PC1                         1   0.05  0.0549   0.131 0.7177  
PC2                         1   0.43  0.4314   1.032 0.3115  
Residuals                 137  57.28  0.4181                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table     1   0.06  0.0557   0.410 0.5222  
cvs_graph     1   0.24  0.2397   1.766 0.1845  
variable      3   1.35  0.4512   3.325 0.0197 *
sim_index     1   0.24  0.

As expected, CVS doesn't predict quant transfer scores.

Post hoc by variable and similar sim (weither students had worked with a similar sim before) coming soon.

## Stat model 3: Predicting the use of CVS

We need to use GEE (generalized estimating equations) for our logistical model with repeated measures.

**coming soon!**

In [40]:
load(gee)

ERROR: Error in load(gee): object 'gee' not found


In [ ]:
gm1 <- gee(cvs_table ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, id=sid, data=mydata)
summary(gm1)

In [ ]:
gm2 <- gee(cvs_graph ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2, id=sid, data=mydata)
summary(gm2)